Analysis

New ideas
* Dirac neutrino produces along DM at LHC
* Monopologes from PBH https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.50.1734
* Heavy DM with Light gauge boson through light DM
* 

In [183]:
import itertools
import sys
import numpy as np
def get_permutations(l,n=3):
    prmts=[]
    for p in itertools. permutations(l, n):
        if sorted(p) not in prmts:
            prmts.append(sorted(p))
    return [ np.asarray(p) for p in prmts if isinstance(p,list) ]
def jsonKeys2int(x):
    if isinstance(x, dict):
            return {int(k):v for k,v in x.items()}
    return x

In [184]:
import pandas as pd

In [185]:
cl=pd.read_json('cl.json').reset_index(drop=True)
cl['sltn']=cl['sltn'].apply(jsonKeys2int)
cl=cl.sort_values('n').reset_index(drop=True)
cl6=pd.read_json('cl6.json').reset_index(drop=True)
cl6['sltn']=cl6['sltn'].apply(jsonKeys2int)
cl6=cl6.sort_values('n').reset_index(drop=True)

DD analysis

In [186]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [187]:
def extract(l,x,νmax=5):
    for i in range(νmax):
        try:
            l.remove(x)
        except:
            break
    return l

def remove_list(l,pr):
    for x in pr:
        try:
            l=extract(l,x)
        except:
            pass
    return l

def remove(l,pr):
    if pr:
        pr=np.concatenate(pr).ravel()
    l=remove_list(l,pr)
    return l

def get_massless_fermions(l,ν,m=0,Dim=5):
    '''
    Zero determinant check implemented.
    '''
    if Dim==5:
        d=1
    elif Dim==6:
        d=2
    s=-(ν+m)//d
    l=extract(l,ν)
    lold=l.copy()
    if m:
        l=extract(l,m)
    #Dirac masses
    det0=0
    pr=[ p for  p in get_permutations( l,n=2  ) if np.append( p,s).sum()==0 or np.append( p,-s).sum()==0 ]
    #print(pr,s)
    for i in range(len(pr)):
        if pr and sum( [ lold.count(x) for x in pr[i] ] )%2!=0:
            det0=det0+1
        l=remove(l,pr)
        if det0!=0:
            l.append(-1000)

    #Majorana masses
    pr=[ll for ll in l if 2*ll+s==0 or 2*ll-s==0]
    l=remove_list(l,pr)
    #print(pr,s)
    #If repeated an extra S' gives at least degenerate Majorana mass
    return list(np.unique(l))
    #return l


def count_massless_fermions(l,ν,m=0,Dim=5):
    return len( get_massless_fermions(l,ν,m,Dim=Dim) )

def get_label(model,Dim=5):
    if Dim==5:
        dd=''
        dm=1
    elif Dim==6:
        dd='2'
        dm=2    
    if model=='DarkDirac':
        label='D→nu_R+{}fi+{}fj'.format(dd,dd)
    elif model=='DarkMajor':
        label='D→nu_R+{}fi'.format(2*dm)
    elif model=='XDirac':
        label='X→nu_R+{}fi+{}fj'.format(dd,dd)
    else:
        label='X→nu_R+{}fi'.format(2*dm)
    return model,label

def sltn_intersection(row,model='DarkDirac',Dim=5):
    model,label=get_label(model,Dim=Dim)
    sltn_int=0
    l=list(itertools.chain(*row['sltn'].values() ) )
    #print(l,label)
    for d in row['nu_R']:
        if d.get(label):
            #print(d)
            intrs=len(np.intersect1d(d.get(label), l ))
            if intrs>sltn_int:
                sltn_int=intrs
    return sltn_int

In [188]:
assert get_massless_fermions([2, -3, -3, -5, 9, -13, 15, 16, -18],ν=-3)==[9]
assert count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=0)==2
assert count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=-5)==1
assert count_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13)==0
assert count_massless_fermions( [5, 13, 13, -16, -18, -23, 26],ν=13)==3
assert get_massless_fermions( [2, -4, 7, 9, -18, -18, 22],ν=-18 )==[2,7]
# XDirac with two triplets and Dim 6
assert get_massless_fermions([2, -7, -8, -8, -8, 18, 18, 18, -25],-8,m=18,Dim=6)==[-25]
#Check determinant zero mass matrix
assert get_massless_fermions( [5, 5, 7, -8, -12, -12, 15],ν=-12,m=0,Dim=5  )==[-1000, -8, 15]
assert get_massless_fermions([2, 4, 4, -15, -17, 26, 26, -30],ν=26,m=0,Dim=6)==[-1000, -30]
assert get_massless_fermions([1, 1, 2, -3, 4, -6, -7, 8],ν=1,m=0,Dim=6)==[-6]
assert get_massless_fermions([1, 2, -6, -6, -6, 8, 9, 9, -11],ν=9,m=0,Dim=5)==[-6] # 6 is ONE massless degenerate, s' give all of the same mass
assert get_massless_fermions([1, 1, 2, 3, 4, -6, -6, -10, 11],ν=-6,m=0,Dim=5)==[1, 11] # Two Dirac and one Majorana

In [189]:
def massfer(row,label='DarkDirac',nmax=10,Dim=5):        
    # 
    if label=='DarkDirac' or label=='DarkMajor':
        if label=='DarkDirac':
            nuR='DD'
        else:
            nuR='DM'
        if row.get(label)>0:
            cdd=[]
            for ν in row.get(nuR):
                l=row.get('solution').copy()
                cdd.append( count_massless_fermions(l , ν,Dim=Dim)  )
            return min(cdd)
        else:
            return nmax
    if label=='XDirac' or label=='XMajor':
        if label=='XDirac':
            nuR='XD'
        else:
            nuR='XM'
        if row.get(label)>0:
            cddmax=100
            for m in row['sltn'].get(3):
                cdp=[]
                for ν in row.get(nuR):
                    l=row.get('solution').copy()
                    l=[ll for ll in l if ll!=m]
                    cdp.append( count_massless_fermions(l , ν,m,Dim=Dim)  )
                if min(cdp)<cddmax:
                    cdd=cdp
            return min(cdd)
        else:
            return nmax    

In [190]:
cl['DD_massless']=cl.apply(lambda row:   massfer(row,label='DarkDirac'),axis='columns')
cl['DM_massless']=cl.apply(lambda row:   massfer(row,label='DarkMajor'),axis='columns')
cl['XD_massless']=cl.apply(lambda row:   massfer(row,label='XDirac'   ),axis='columns')
cl['XM_massless']=cl.apply(lambda row:   massfer(row,label='XMajor'   ),axis='columns')

In [191]:
cl6['DD_massless']=cl6.apply(lambda row: massfer(row,label='DarkDirac',Dim=6),axis='columns')
cl6['DM_massless']=cl6.apply(lambda row: massfer(row,label='DarkMajor',Dim=6),axis='columns')
cl6['XD_massless']=cl6.apply(lambda row: massfer(row,label='XDirac'   ,Dim=6),axis='columns')
cl6['XM_massless']=cl6.apply(lambda row: massfer(row,label='XMajor'   ,Dim=6),axis='columns')

In [192]:
z=np.array([1,1,1,1,1,1,-4,-4,-4,-4,5,5])
z.sum(),(z**3).sum()

(0, 0)

In [193]:
cl['total']   =cl['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )
#cl['sltn_len']=-cl['sltn'].apply(lambda d: list(itertools.chain(*d.values()))).apply(len)

In [194]:
cl6['total']   =cl6['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )
#cl6['sltn_len']=-cl6['sltn'].apply(lambda d: list(itertools.chain(*d.values()))).apply(len)

In [195]:
row=cl[cl['solution'].astype(str)=='[1, 4, 5, 5, -8, -10, -10, 13]'].reset_index(drop=True).loc[0].to_dict()

In [196]:
sltn_intersection(row,model='DarkMajor')

2

## Filter unique solutions

Pure solutions

In [197]:
dim=[5,6]
#d=dim[0]
for d in dim:
    if d==5:
        fl=pd.DataFrame()
        clc=cl.copy()
    else:
        fl6=pd.DataFrame()
        clc=cl6.copy()
    models=pd.DataFrame([
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']>0))},
         {'model':'DarkDirac','condition':((clc['DarkDirac']>0)  & (clc['DarkMajor']>0)  & (clc['XDirac']>0)  & (clc['XMajor']>0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']> 0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']==0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']==0) & (clc['XMajor']> 0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']==0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']> 0))},
         {'model':'DarkMajor','condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']>0)  & (clc['XDirac']> 0) & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']==0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XDirac'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']>0)  & (clc['XMajor']> 0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']==0) & (clc['XDirac']> 0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']==0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']==0) & (clc['XDirac']> 0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']==0) & (clc['DarkMajor']> 0) & (clc['XDirac']> 0) & (clc['XMajor']>0))},
         {'model':'XMajor'   ,'condition':((clc['DarkDirac']> 0) & (clc['DarkMajor']> 0) & (clc['XDirac']> 0) & (clc['XMajor']>0))}
    ])
    #i=0
    for i in models.index:
        mdl=models.loc[i,'model']
        tmp=clc[models.loc[i,'condition']].reset_index(drop=True)
        if not tmp.empty:
            tmp['sltn_len']=-tmp.apply(lambda row: sltn_intersection(row, model=mdl,Dim=d ) ,axis='columns')
            massless={'DarkDirac':'DD_massless','DarkMajor':'DM_massless','XDirac':'XD_massless','XMajor':'XM_massless'}
            if d==5:
                fl=fl.append(
                    tmp.sort_values(['n',massless[mdl],'sltn_len','total']
                     ).drop_duplicates(['n',massless[mdl],'sltn_len']).reset_index(drop=True)
                ).reset_index(drop=True)
                fl['str']=fl['solution'].astype(str)
                fl=fl.drop_duplicates('str').drop('str',axis='columns').reset_index(drop=True)                 
            if d==6:
                fl6=fl6.append(
                    tmp.sort_values(['n',massless[mdl],'sltn_len','total']
                     ).drop_duplicates(['n',massless[mdl],'sltn_len']).reset_index(drop=True)
                ).reset_index(drop=True)
                fl6['str']=fl6['solution'].astype(str)
                fl6=fl6.drop_duplicates('str').drop('str',axis='columns').reset_index(drop=True)                                 

In [198]:
cl[cl['solution'].astype(str)=='[5, 5, 7, -8, -12, -12, 15]']

,n,l,k,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total
87,7,"[-12, -13]","[-10, 4, -10]","[5, 5, 7, -8, -12, -12, 15]",948,"[{'D→nu_R+fi+fj': [-12, 5, 7]}]",1,0,0,0,"{2: [5, -12]}","[-12, 5]",0,0,0,3,10,10,10,64


In [199]:
fl=fl[fl.apply(lambda r:min([r['DD_massless'],r['DM_massless'],r['XM_massless'],r['XD_massless']])<=2,axis='columns' )
     ].sort_values(['n',massless[mdl],'sltn_len','total'])[
       ['n','l','k','gcd','solution','DD', 'DM', 'XD', 'XM', 'DD_massless',
       'DM_massless','XD_massless', 'XM_massless','total','sltn_len','nu_R']].reset_index(drop=True)
fl

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
0,6,"[-21, -14]","[-14, -21]",36015,"[1, -2, -3, 5, 5, -6]",[5],0,0,0,0,10,10,10,22,-1,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]"
1,7,"[-13, 0]","[-13, 13, -13]",28561,"[1, 2, 2, -3, -3, -3, 4]",[-3],0,0,0,2,10,10,10,18,-2,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]"
2,7,"[-12, -10]","[-6, -3, -2]",264,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,2,2,10,10,28,-2,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]"
3,7,"[-12, -9]","[6, -2, -6]",144,"[2, 3, 3, -4, -5, -6, 7]",[3],0,0,0,1,10,10,10,30,-1,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]"
4,7,"[-12, -10]","[-12, -9, -5]",852,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2,2,10,10,32,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]"
5,7,"[-12, -9]","[3, -6, 3]",648,"[1, 1, -4, -4, 7, 8, -9]",[1],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]"
6,8,"[-12, -8, -12]","[3, -3, -9]",5328,"[1, 1, 2, 3, -4, -4, -5, 6]","[-4, 1]",[-4],0,0,2,2,10,10,26,-2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]"
7,8,"[-12, 6, -12]","[-12, -8, 0]",3456,"[2, 4, 4, 5, -9, -9, -10, 13]","[-9, 4]",0,0,0,2,10,10,10,56,-2,"[{'D→nu_R+fi+fj': [-9, 4, 5]}]"
8,8,"[-12, -9, 3]","[4, 2, -4]",1260,"[1, 2, 2, 2, -3, -5, -6, 7]",[2],0,0,0,1,10,10,10,28,-1,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]"
9,8,"[-12, -9, 3]","[-3, -12, -6]",972,"[1, 2, 2, 4, -5, -5, -7, 8]",[-5],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-5, 1, 4]}]"


In [200]:
np.sort( fl['solution'].apply(lambda l: np.abs(l).max()).unique() ).max()

22

In [201]:
fl6=fl6[fl6.apply(lambda r:min([r['DD_massless'],r['DM_massless'],r['XM_massless'],r['XD_massless']])<=2,axis='columns' )
       ].sort_values(['n',massless[mdl],'sltn_len','total'])[
       ['n','l','k','gcd','solution','DD', 'DM', 'XD', 'XM', 'DD_massless',
       'DM_massless','XD_massless', 'XM_massless','total','sltn_len','nu_R']].reset_index(drop=True)

In [202]:
fl6

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
0,6,"[-20, -10]","[5, 10]",15000,"[1, 1, 1, -4, -4, 5]",0,[-4],0,0,10,1,10,10,16,-2,"[{'D→nu_R+4fi': [-4, 1]}]"
1,6,"[-21, 21]","[-21, -14]",43218,"[1, -4, -4, 9, 9, -11]",0,[-4],0,0,10,1,10,10,38,-1,"[{'D→nu_R+4fi': [-4, 1]}]"
2,6,"[-21, 14]","[-21, 3]",21168,"[1, -4, -8, 14, 14, -17]",[14],0,0,0,2,10,10,10,58,-1,"[{'D→nu_R+2fi+2fj': [-8, 1, 14]}]"
3,7,"[-12, -9]","[3, -6, 3]",648,"[1, 1, -4, -4, 7, 8, -9]",0,[-4],0,0,10,1,10,10,34,-2,"[{'D→nu_R+4fi': [-4, 1]}]"
4,7,"[-12, -9]","[-12, -11, -2]",351,"[2, 2, -4, 7, -8, -8, 9]",[2],[-8],0,0,2,3,10,10,40,-2,"[{'D→nu_R+2fi+2fj': [-8, 2, 7]}, {'D→nu_R+4fi': [-8, 2]}]"
5,7,"[-12, -10]","[-12, -9, -5]",852,"[1, 3, -4, 5, -6, -6, 7]",[-6],0,0,0,2,10,10,10,32,-1,"[{'D→nu_R+2fi+2fj': [-6, -4, 7]}]"
6,7,"[-12, -9]","[-6, -2, 6]",504,"[3, 3, 3, -5, -5, -7, 8]",0,0,[-5],0,10,10,0,10,34,-1,"[{'X→nu_R+2fi+2fj': [-7, -5, 8]}]"
7,7,"[-12, -9]","[3, -2, 7]",108,"[4, 4, 5, -7, -8, -9, 11]",[4],0,0,0,1,10,10,10,48,-1,"[{'D→nu_R+2fi+2fj': [-7, 4, 5]}]"
8,7,"[-12, 3]","[-6, -7, 6]",135,"[7, 10, -18, 25, -28, -28, 32]",[-28],[-28],0,0,2,2,10,10,148,-1,"[{'D→nu_R+4fi': [-28, 7]}, {'D→nu_R+2fi+2fj': [-28, -18, 32]}]"
9,8,"[-12, -8, -12]","[-4, -12, 4]",4096,"[1, 3, 3, 3, -5, -7, -7, 9]",0,0,0,[-7],10,10,10,2,38,-1,"[{'X→nu_R+4fi': [-7, 1]}]"


In [203]:
np.sort( fl6['solution'].apply(lambda l: np.abs(l).max()).unique() ).max()

32

In [204]:
n=9
x=1
fl6[(fl6['n']==n) & (fl6['XD_massless']==x)].sort_values(['DD_massless','DM_massless','XM_massless'],ascending=False)

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
45,9,"[-6, -7, -2]","[6, -2, -1, -5]",158,"[4, 4, 4, 5, -6, -6, -6, -10, 11]",0,0,"[4, -6]",0,10,10,1,10,56,-1,"[{'X→nu_R+2fi+2fj': [-10, -6, 11]}, {'X→nu_R+2fi+2fj': [-10, 4, 11]}]"
50,9,"[18, 13, 6]","[-10, 6, 1, -4]",608,"[2, -7, -8, -8, -8, 18, 18, 18, -25]",0,[-8],"[-8, 18]",0,10,3,1,10,112,-1,"[{'D→nu_R+4fi': [-8, 2]}, {'X→nu_R+2fi+2fj': [-7, 2, 18]}, {'X→nu_R+2fi+2fj': [-8, -7, 2]}]"
39,9,"[-7, 0, -2]","[-4, 4, -5, -2]",46,"[4, 7, -8, 9, -16, -16, -16, 18, 18]",[18],[-16],[18],0,3,4,1,10,112,-2,"[{'D→nu_R+2fi+2fj': [-16, 7, 18]}, {'D→nu_R+4fi': [-16, 4]}, {'X→nu_R+2fi+2fj': [-8, 7, 18]}]"


In [205]:
np.sort( fl6['solution'].apply(lambda l: np.abs(l).max()).unique() ).max()

32

In [206]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldsymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [207]:
kk=fl.copy()
#kk=fl6.copy()
for i in kk.index:
#if True:
#    i=6
    for s in ['DD','DM','XD','XM']:
        if kk.loc[i,'{}_massless'.format(s)]==0:
            kk.loc[i,'n']=add_boldsymbol(kk.loc[i,'n'])
            kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])    
#kk=fl[['n','l','k','solution','n','DD','DM','XD','XM']]
kkk=kk[['n','l','k','solution','gcd','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

In [208]:
kk

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
0,\boldsymbol{6},"[-21, -14]","[-14, -21]",36015,"[1, -2, -3, 5, 5, -6]",[\boldsymbol{5}],0,0,0,0,10,10,10,22,-1,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]"
1,7,"[-13, 0]","[-13, 13, -13]",28561,"[1, 2, 2, -3, -3, -3, 4]",[-3],0,0,0,2,10,10,10,18,-2,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]"
2,7,"[-12, -10]","[-6, -3, -2]",264,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,2,2,10,10,28,-2,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]"
3,7,"[-12, -9]","[6, -2, -6]",144,"[2, 3, 3, -4, -5, -6, 7]",[3],0,0,0,1,10,10,10,30,-1,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]"
4,7,"[-12, -10]","[-12, -9, -5]",852,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2,2,10,10,32,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]"
5,7,"[-12, -9]","[3, -6, 3]",648,"[1, 1, -4, -4, 7, 8, -9]",[1],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]"
6,8,"[-12, -8, -12]","[3, -3, -9]",5328,"[1, 1, 2, 3, -4, -4, -5, 6]","[-4, 1]",[-4],0,0,2,2,10,10,26,-2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]"
7,8,"[-12, 6, -12]","[-12, -8, 0]",3456,"[2, 4, 4, 5, -9, -9, -10, 13]","[-9, 4]",0,0,0,2,10,10,10,56,-2,"[{'D→nu_R+fi+fj': [-9, 4, 5]}]"
8,8,"[-12, -9, 3]","[4, 2, -4]",1260,"[1, 2, 2, 2, -3, -5, -6, 7]",[2],0,0,0,1,10,10,10,28,-1,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]"
9,8,"[-12, -9, 3]","[-3, -12, -6]",972,"[1, 2, 2, 4, -5, -5, -7, 8]",[-5],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-5, 1, 4]}]"


In [209]:
n=r'\boldsymbol{9}'
x=0
kk[(kk['n']==n) & (kk['DD_massless']==x)]

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R
17,\boldsymbol{9},"[-6, 0, 3]","[-6, 6, 0, -6]",324,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",[\boldsymbol{9}],0,0,[1],0,10,10,2,58,-3,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]"
20,\boldsymbol{9},"[-6, 0, -2]","[2, -3, 0, 3]",24,"[1, 1, -3, -8, 11, 11, 11, -12, -12]",[\boldsymbol{11}],0,0,0,0,10,10,10,70,-3,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'D→nu_R+fi+fj': [-8, -3, 11]}]"
29,\boldsymbol{9},"[-7, -5, -6]","[-7, -6, -4, -2]",74,"[1, -2, 3, 4, 6, -7, -7, -7, 9]",[\boldsymbol{-7}],0,0,0,0,10,10,10,46,-1,"[{'D→nu_R+fi+fj': [-7, -2, 9]}, {'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+fi+fj': [-7, 3, 4]}]"
31,\boldsymbol{9},"[-7, -4, -6]","[1, -2, 4, -1]",106,"[1, -2, -2, -2, 5, -7, 8, 9, -10]",[\boldsymbol{-2}],[\boldsymbol{-2}],0,0,0,0,10,10,46,-1,"[{'D→nu_R+2fi': [-2, 1]}, {'D→nu_R+fi+fj': [-7, -2, 9]}]"


## Manual analysis

In [339]:
def extract_multiplets(l):
    xc=[]
    xr=0
    for x in l:
        if l.count(x)>1:
            if xr!=x:
                xc.append((x,l.count(x)))
                xr=x
    return xc
def extra_multiplets(row,slt='DD'):
    extramax=0
    for ν in row.get(slt):
        extra=0
        l=row.get('solution').copy()
        if l.count(ν)>2:
            extra=extra+0.25
        l=extract(l,ν)
        if slt=='XD' or slt=='XM':
            #Remove SM charges
            ms=[ x[0] for x in extract_multiplets(l) if x[-1]==3]
            if not ms:
                sys.exit('Bad X-solution → {}'.format(row['solution']))
            if len(ms)>2:
                sys.exit('NO 3 triplets yet implemented')
            l=extract(l,ms[0])
        em=extract_multiplets(l)            
        extra=extra+sum( [ x[-1] for x in em ] )
        if extra>extramax:
                extramax=extra
    return extramax

def get_score(fl,s='DD_massless',i=1):
    tmp=fl[fl[s]==i].reset_index(drop=True)
    if not tmp.empty:
        tmp['score']=tmp.apply(lambda row: extra_multiplets(row,slt=s.split('_')[0]),axis='columns')
    return tmp



def get_unstable(row):
    snu={'D→nu_R+fi+fj'  :'DD','D→nu_R+2fi':'DM','X→nu_R+fi+fj'  :'XD','X→nu_R+2fi':'XM',
         'D→nu_R+2fi+2fj':'DD','D→nu_R+4fi':'DM','X→nu_R+2fi+2fj':'XD','X→nu_R+4fi':'XM'}    
    for d in row['nu_R']:
        k=list(d.keys())[0]
        for ν in row.get( snu[k] ):
            ls=row['solution'].copy()
            l=list(d.values())[0].copy()
            #print(ν,l,ls)
            l=extract(l,ν)
            ls=extract(ls,ν)
            #print( "***",l,ls )
            if k.find('X→')>-1:
                m=[ x for x in ls if ls.count(x)==3][0]
                ls=extract(ls,m)
                #print('ls',ls)                
            if len(l)==2:
                s=sum(l)                    
                if 0 in np.array(ls)+ν-s or 0 in np.array(ls)+ν+s:
                    return True
            elif len(l)==1:
                s=sum(l)*2                    
                if 0 in np.array(ls)+ν-s or 0 in np.array(ls)+ν+s:
                    return True                   
            else:
                print('WARNING bad solition in {}'.format(row['solution']))
    return False        

In [388]:
assert extra_multiplets({'solution':[2,2,2,3,3,3,4,4,4,8,8],'XD':[2]},slt='XD')==5.25
assert extra_multiplets({'solution':[2,2,2,3,3,3,4,4,4,8,8],'DD':[2]},slt='DD')==8.25
assert extra_multiplets({'solution':[2, 2, 2, -3, -3, 4, -5, -5, 6],'XD':[-5,-3]},slt='XD')==2

In [389]:
assert not get_unstable(fl[fl['solution'].astype(str)=='[4, 4, 4, -5, -9, -10, -10, 11, 11]'].iloc[-1])
assert not get_unstable(fl[fl['solution'].astype(str)=='[2, -5, -7, -7, 10, 10, 10, -13]'].iloc[-1])
assert get_unstable(fl6[fl6['solution'].astype(str)=='[2, 2, -3, 5, -6, 7, -8, -9, 10]'].iloc[-1])

In [390]:
fl['unstable']=fl.apply( lambda row: get_unstable(row) ,axis='columns')
fl6['unstable']=fl6.apply( lambda row: get_unstable(row) ,axis='columns')

WARNING bad solition in [4, 4, -9, 11, -13, -13, 18, 18, -20]
WARNING bad solition in [4, 4, 4, 5, -6, -6, -6, -10, 11]
WARNING bad solition in [2, -7, -8, -8, -8, 18, 18, 18, -25]
WARNING bad solition in [2, -7, -8, -8, -8, 18, 18, 18, -25]


In [471]:
fl['Q']=1
i=0
iq=fl[fl['solution'].astype(str)=='[1, 1, -3, -8, 11, 11, 11, -12, -12]'].index[0]
fl.loc[iq,'Q']=0
get_score(fl,s='DD_massless',i=i).sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
0,6,"[-21, -14]","[-14, -21]",36015,"[1, -2, -3, 5, 5, -6]",[5],0,0,0,0,10,10,10,22,-1,"[{'D→nu_R+fi+fj': [-6, 1, 5]}, {'D→nu_R+fi+fj': [-3, -2, 5]}]",1,False,0.00
4,9,"[-7, -4, -6]","[1, -2, 4, -1]",106,"[1, -2, -2, -2, 5, -7, 8, 9, -10]",[-2],[-2],0,0,0,0,10,10,46,-1,"[{'D→nu_R+2fi': [-2, 1]}, {'D→nu_R+fi+fj': [-7, -2, 9]}]",1,False,0.25
1,9,"[-6, 0, 3]","[-6, 6, 0, -6]",324,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",[9],0,0,[1],0,10,10,2,58,-3,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",1,False,4.25
2,9,"[-6, 0, -2]","[2, -3, 0, 3]",24,"[1, 1, -3, -8, 11, 11, 11, -12, -12]",[11],0,0,0,0,10,10,10,70,-3,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'D→nu_R+fi+fj': [-8, -3, 11]}]",0,False,4.25
3,9,"[-7, -5, -6]","[-7, -6, -4, -2]",74,"[1, -2, 3, 4, 6, -7, -7, -7, 9]",[-7],0,0,0,0,10,10,10,46,-1,"[{'D→nu_R+fi+fj': [-7, -2, 9]}, {'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+fi+fj': [-7, 3, 4]}]",1,False,0.25


In [472]:
get_score(fl,s='DM_massless',i=i).sort_values(['n','DD_massless','XD_massless','XM_massless'])   

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
0,9,"[-7, -4, -6]","[1, -2, 4, -1]",106,"[1, -2, -2, -2, 5, -7, 8, 9, -10]",[-2],[-2],0,0,0,0,10,10,46,-1,"[{'D→nu_R+2fi': [-2, 1]}, {'D→nu_R+fi+fj': [-7, -2, 9]}]",1,False,0.25


In [473]:
iq=fl[fl['solution'].astype(str)=='[5, 6, 6, 6, -8, -9, -9, -10, 13]'].index[0]
fl.loc[iq,'Q']=0
get_score(fl,s='XD_massless',i=i).sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
0,8,"[-12, -8, -12]","[-4, -12, 4]",4096,"[1, 3, 3, 3, -5, -7, -7, 9]",0,0,[-7],0,10,10,0,10,38,-1,"[{'X→nu_R+fi+fj': [-7, -5, 9]}]",1,False,0
1,9,"[-6, -4, -6]","[2, -7, -3, -6]",60,"[1, 2, -6, -6, -6, 8, 9, 9, -11]",[9],0,[9],0,1,10,0,10,58,-1,"[{'D→nu_R+fi+fj': [-11, 2, 9]}, {'X→nu_R+fi+fj': [-11, 8, 9]}]",1,False,0
2,9,"[-6, -4, 0]","[-4, -3, 3, 2]",48,"[5, 6, 6, 6, -8, -9, -9, -10, 13]",0,0,[-9],0,10,10,0,10,72,-1,"[{'X→nu_R+fi+fj': [-10, -9, 13]}]",0,False,0


In [474]:
get_score(fl,s='XM_massless',i=i).sort_values(['n','DD_massless','DM_massless','XD_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable


In [475]:
#fl['Q']=1
iq=fl[fl['solution'].astype(str)=='[2, 3, 3, -4, -5, -6, 7]'].index[0]
fl.loc[iq,'Q']=0
i=1
get_score(fl,s='DD_massless',i=i).sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
0,7,"[-12, -9]","[6, -2, -6]",144,"[2, 3, 3, -4, -5, -6, 7]",[3],0,0,0,1,10,10,10,30,-1,"[{'D→nu_R+fi+fj': [-5, 2, 3]}]",0,True,0.00
2,8,"[-12, -9, 3]","[-11, -8, -9]",540,"[3, -4, -5, 8, 8, -11, -12, 13]",[8],[8],0,0,1,1,10,10,64,-1,"[{'D→nu_R+2fi': [8, -4]}, {'D→nu_R+fi+fj': [-11, 3, 8]}]",1,False,0.00
1,8,"[-12, -9, 3]","[4, 2, -4]",1260,"[1, 2, 2, 2, -3, -5, -6, 7]",[2],0,0,0,1,10,10,10,28,-1,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,False,0.25
7,9,"[-7, -6, 2]","[4, 0, -6, -4]",24,"[2, -3, 4, 6, 6, -7, -10, -11, 13]",[6],[6],0,0,1,1,10,10,62,-1,"[{'D→nu_R+2fi': [6, -3]}, {'D→nu_R+fi+fj': [-10, 4, 6]}]",1,False,0.00
4,9,"[-6, -4, -7]","[-5, -2, 1, -3]",85,"[1, 1, 2, 2, 3, -5, -6, -6, 8]",[2],[-6],0,0,1,4,10,10,34,-1,"[{'D→nu_R+fi+fj': [-5, 2, 3]}, {'D→nu_R+2fi': [-6, 3]}]",1,False,4.00
6,9,"[-6, -4, -6]","[2, -7, -3, -6]",60,"[1, 2, -6, -6, -6, 8, 9, 9, -11]",[9],0,[9],0,1,10,0,10,58,-1,"[{'D→nu_R+fi+fj': [-11, 2, 9]}, {'X→nu_R+fi+fj': [-11, 8, 9]}]",1,False,3.00
3,9,"[-6, 4, -2]","[4, 6, 4, -2]",48,"[1, -3, -3, -3, -5, 8, 8, 8, -11]","[-3, 8]",0,0,0,1,10,10,10,50,-2,"[{'D→nu_R+fi+fj': [-5, -3, 8]}]",1,False,3.25
5,9,"[-7, -6, -4]","[4, -4, -1, -7]",140,"[2, -3, -3, -3, -5, 7, 7, 8, -10]",[-3],0,0,0,1,10,10,10,48,-1,"[{'D→nu_R+fi+fj': [-5, -3, 8]}]",1,False,2.25


In [476]:
get_score(fl,s='DM_massless',i=i).sort_values(['n','DD_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
0,8,"[-12, -9, 3]","[-11, -8, -9]",540,"[3, -4, -5, 8, 8, -11, -12, 13]",[8],[8],0,0,1,1,10,10,64,-1,"[{'D→nu_R+2fi': [8, -4]}, {'D→nu_R+fi+fj': [-11, 3, 8]}]",1,False,0
1,9,"[-7, -6, 2]","[4, 0, -6, -4]",24,"[2, -3, 4, 6, 6, -7, -10, -11, 13]",[6],[6],0,0,1,1,10,10,62,-1,"[{'D→nu_R+2fi': [6, -3]}, {'D→nu_R+fi+fj': [-10, 4, 6]}]",1,False,0
2,9,"[-4, 1, -2]","[-1, -2, 4, 1]",2,"[1, -2, -2, -3, -3, -3, 14, 20, -22]",0,[-2],0,0,10,1,10,10,70,-1,"[{'D→nu_R+2fi': [-2, 1]}]",1,False,3


In [477]:
iq=fl[fl['solution'].astype(str)=='[2, -5, -7, -7, 10, 10, 10, -13]'].index[0]
fl.loc[iq,'Q']=0
iq=fl[fl['solution'].astype(str)=='[2, 2, 2, -3, -3, 4, -5, -5, 6]'].index[0]
fl.loc[iq,'Q']=0
get_score(fl,s='XD_massless',i=i).sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
0,8,"[-12, -4, 8]","[-3, 12, 9]",3456,"[2, -5, -5, -5, 7, 8, 8, -10]",[8],0,[8],0,2,10,1,10,50,-1,"[{'D→nu_R+fi+fj': [-10, 2, 8]}, {'X→nu_R+fi+fj': [-10, 7, 8]}]",1,False,0
2,8,"[-12, 0, -12]","[-12, -6, -12]",5184,"[2, -5, -7, -7, 10, 10, 10, -13]",0,[10],[-7],0,10,3,1,10,64,-1,"[{'D→nu_R+2fi': [10, -5]}, {'X→nu_R+fi+fj': [-7, -5, 2]}]",0,False,0
1,8,"[-11, 0, -11]","[-1, 1, 5]",968,"[1, 1, 1, -5, -7, 11, 11, -13]",0,0,[11],0,10,10,1,10,50,-1,"[{'X→nu_R+fi+fj': [-7, -5, 11]}]",1,False,0
3,9,"[-6, -7, -6]","[-3, -4, -1, -5]",85,"[2, 2, 2, -3, -3, 4, -5, -5, 6]",0,0,"[-5, -3]",0,10,10,1,10,32,-2,"[{'X→nu_R+fi+fj': [-5, -3, 6]}]",0,True,2
4,9,"[-7, -6, 4]","[3, -1, 5, -2]",18,"[4, 4, 6, 6, -7, -7, -7, -12, 13]",0,0,[6],0,10,10,1,10,66,-1,"[{'X→nu_R+fi+fj': [-12, 6, 13]}]",1,False,2


In [478]:
get_score(fl,s='XM_massless',i=i).sort_values(['n','DD_massless','DM_massless','XD_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable


In [479]:
i=2
iq=fl[fl['solution'].astype(str)=='[1, 1, -4, -4, 7, 8, -9]'].index[0]
fl.loc[iq,'Q']=0
iq=fl[fl['solution'].astype(str)=='[2, 4, 4, 5, -9, -9, -10, 13]'].index[0]
fl.loc[iq,'Q']=0
get_score(fl,s='DD_massless',i=i).sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
1,7,"[-12, -10]","[-6, -3, -2]",264,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,2,2,10,10,28,-2,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1,False,2.00
2,7,"[-12, -10]","[-12, -9, -5]",852,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2,2,10,10,32,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,False,0.00
0,7,"[-13, 0]","[-13, 13, -13]",28561,"[1, 2, 2, -3, -3, -3, 4]",[-3],0,0,0,2,10,10,10,18,-2,"[{'D→nu_R+fi+fj': [-3, 1, 2]}]",1,False,2.25
3,7,"[-12, -9]","[3, -6, 3]",648,"[1, 1, -4, -4, 7, 8, -9]",[1],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-9, 1, 8]}]",0,False,2.00
4,8,"[-12, -8, -12]","[3, -3, -9]",5328,"[1, 1, 2, 3, -4, -4, -5, 6]","[-4, 1]",[-4],0,0,2,2,10,10,26,-2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]",1,False,2.00
7,8,"[-12, -11, -5]","[4, 12, 4]",768,"[1, 2, 3, 5, -6, -6, -9, 10]",[-6],[-6],0,0,2,2,10,10,42,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,False,0.00
8,8,"[-12, -4, 8]","[-3, 12, 9]",3456,"[2, -5, -5, -5, 7, 8, 8, -10]",[8],0,[8],0,2,10,1,10,50,-1,"[{'D→nu_R+fi+fj': [-10, 2, 8]}, {'X→nu_R+fi+fj': [-10, 7, 8]}]",1,False,3.00
5,8,"[-12, 6, -12]","[-12, -8, 0]",3456,"[2, 4, 4, 5, -9, -9, -10, 13]","[-9, 4]",0,0,0,2,10,10,10,56,-2,"[{'D→nu_R+fi+fj': [-9, 4, 5]}]",0,False,2.00
6,8,"[-12, -9, 3]","[-3, -12, -6]",972,"[1, 2, 2, 4, -5, -5, -7, 8]",[-5],0,0,0,2,10,10,10,34,-1,"[{'D→nu_R+fi+fj': [-5, 1, 4]}]",1,False,2.00
10,9,"[-7, -4, 6]","[-7, -3, -2, 1]",96,"[1, 1, -5, -7, 12, 14, 14, -15, -15]","[-15, 1, 14]",[14],0,0,2,2,10,10,84,-3,"[{'D→nu_R+fi+fj': [-15, 1, 14]}, {'D→nu_R+2fi': [14, -7]}]",1,False,4.00


In [480]:
iq=fl[fl['solution'].astype(str)=='[4, -8, -8, 11, -12, 14, 21, -22]'].index[0]
fl.loc[iq,'Q']=0
iq=fl[fl['solution'].astype(str)=='[1, 1, 2, 3, 4, -6, -6, -10, 11]'].index[0]
fl.loc[iq,'Q']=0
get_score(fl,s='DM_massless',i=i).sort_values(['n','DD_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
0,7,"[-12, -10]","[-6, -3, -2]",264,"[1, 1, -3, -4, 6, 6, -7]","[1, 6]",[6],0,0,2,2,10,10,28,-2,"[{'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+2fi': [6, -3]}]",1,False,2
1,7,"[-12, -10]","[-12, -9, -5]",852,"[1, 3, -4, 5, -6, -6, 7]",[-6],[-6],0,0,2,2,10,10,32,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,False,0
2,8,"[-12, -8, -12]","[3, -3, -9]",5328,"[1, 1, 2, 3, -4, -4, -5, 6]","[-4, 1]",[-4],0,0,2,2,10,10,26,-2,"[{'D→nu_R+fi+fj': [-4, 1, 3]}, {'D→nu_R+2fi': [-4, 2]}]",1,False,2
3,8,"[-12, -11, -5]","[4, 12, 4]",768,"[1, 2, 3, 5, -6, -6, -9, 10]",[-6],[-6],0,0,2,2,10,10,42,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 1, 5]}]",1,False,0
4,8,"[-12, -10, -6]","[4, -7, -1]",516,"[4, -8, -8, 11, -12, 14, 21, -22]",0,[-8],0,0,10,2,10,10,100,-1,"[{'D→nu_R+2fi': [-8, 4]}]",0,False,0
5,9,"[-7, -4, 6]","[-7, -3, -2, 1]",96,"[1, 1, -5, -7, 12, 14, 14, -15, -15]","[-15, 1, 14]",[14],0,0,2,2,10,10,84,-3,"[{'D→nu_R+fi+fj': [-15, 1, 14]}, {'D→nu_R+2fi': [14, -7]}]",1,False,4
6,9,"[-7, -5, -7]","[-6, -4, 5, 2]",89,"[1, -2, -2, -4, 7, -9, 11, 11, -13]","[-2, 11]",[-2],0,0,2,2,10,10,60,-2,"[{'D→nu_R+2fi': [-2, 1]}, {'D→nu_R+fi+fj': [-9, -2, 11]}]",1,False,2
7,9,"[-6, -5, -7]","[-5, 1, 6, 2]",115,"[1, 1, 2, 3, 4, -6, -6, -10, 11]",[-6],[-6],0,0,2,2,10,10,44,-1,"[{'D→nu_R+2fi': [-6, 3]}, {'D→nu_R+fi+fj': [-6, 2, 4]}]",0,False,2
8,9,"[-7, -4, -6]","[1, -7, -4, -1]",16,"[1, -2, -2, 5, -7, -7, 14, 18, -20]",0,[-2],0,0,10,2,10,10,76,-1,"[{'D→nu_R+2fi': [-2, 1]}]",1,False,2


In [481]:
iq=fl[fl['solution'].astype(str)=='[1, -2, -2, -2, 5, -16, 17, 17, -18]'].index[0]
fl.loc[iq,'Q']=0
get_score(fl,s='XD_massless',i=i).sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
3,9,"[-6, -1, -7]","[-1, -5, -4, -5]",8,"[1, -2, -2, -2, 5, -16, 17, 17, -18]",[17],[-2],[17],0,3,4,2,10,80,-1,"[{'D→nu_R+fi+fj': [-18, 1, 17]}, {'D→nu_R+2fi': [-2, 1]}, {'X→nu_R+fi+fj': [-16, 1, 17]}]",0,False,0
1,9,"[-7, -6, -4]","[-6, 6, 0, -4]",204,"[2, -3, 4, 4, 4, -6, -7, -7, 9]",[4],0,[-7],0,3,10,2,10,46,-1,"[{'D→nu_R+fi+fj': [-6, 2, 4]}, {'X→nu_R+fi+fj': [-7, -6, 9]}]",1,False,0
0,9,"[-6, 3, -6]","[3, 0, 3, 2]",90,"[4, 4, 4, -5, -9, -10, -10, 11, 11]",0,0,"[-10, 11]",0,10,10,2,10,68,-2,"[{'X→nu_R+fi+fj': [-10, -5, 11]}]",1,False,2
2,9,"[-6, -1, -5]","[2, 7, 5, -1]",3,"[1, -3, -3, -3, -6, 8, 8, 10, -12]",0,0,[8],0,10,10,2,10,54,-1,"[{'X→nu_R+fi+fj': [-6, 1, 8]}]",1,False,0


In [482]:
iq=fl[fl['solution'].astype(str)=='[1, 13, 13, -15, -15, -15, -16, 17, 17]'].index[0]
fl.loc[iq,'Q']=0
get_score(fl,s='XM_massless',i=i).sort_values(['n','DD_massless','DM_massless','XD_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable,score
0,9,"[-6, 0, 3]","[-6, 6, 0, -6]",324,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",[9],0,0,[1],0,10,10,2,58,-3,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",1,False,2
1,9,"[-6, 6, -6]","[6, 0, -6, -3]",324,"[1, 13, 13, -15, -15, -15, -16, 17, 17]",0,0,0,[13],10,10,10,2,122,-1,"[{'X→nu_R+2fi': [13, 1]}]",0,False,2


In [483]:
fl.value_counts('Q')

Q
1    31
0    11
dtype: int64

D-6

In [484]:
i=0
fl6['Q']=1
iq=fl6[fl6['solution'].astype(str)=='[1, -3, 4, 4, -6, 8, -9, -10, 11]'].index[0]
fl6.loc[iq,'Q']=0
get_score(fl6,s='DD_massless',i=i).sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q,score
0,9,"[-6, -7, -6]","[-1, -2, 1, 2]",12,"[1, -3, 4, 4, -6, 8, -9, -10, 11]",[4],0,0,0,0,10,10,10,56,-1,"[{'D→nu_R+2fi+2fj': [-10, 4, 8]}, {'D→nu_R+2fi+2fj': [-3, 1, 4]}]",True,0,0


In [485]:
get_score(fl6,s='DM_massless',i=i).sort_values(['n','DD_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q


In [486]:
iq=fl6[fl6['solution'].astype(str)=='[8, 8, 8, -12, -12, 15, -17, -23, 25]'].index[0]
fl6.loc[iq,'Q']=0
get_score(fl6,s='XD_massless',i=i).sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q,score
0,7,"[-12, -9]","[-6, -2, 6]",504,"[3, 3, 3, -5, -5, -7, 8]",0,0,[-5],0,10,10,0,10,34,-1,"[{'X→nu_R+2fi+2fj': [-7, -5, 8]}]",False,1,0
1,9,"[-3, 1, -3]","[-4, 5, -4, 3]",4,"[1, -3, 8, 8, 8, -12, -12, -17, 19]",0,0,[-12],[-12],10,10,0,0,88,-1,"[{'X→nu_R+4fi': [-12, 1]}, {'X→nu_R+2fi+2fj': [-17, -12, 19]}]",False,1,0
2,9,"[-4, -7, 4]","[-1, 1, 5, 3]",36,"[8, 8, 8, -12, -12, 15, -17, -23, 25]",0,0,[-12],0,10,10,0,10,128,-1,"[{'X→nu_R+2fi+2fj': [-23, -12, 25]}]",False,0,0


In [487]:
get_score(fl,s='XM_massless',i=i).sort_values(['n','DD_massless','DM_massless','XD_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,Q,unstable


In [488]:
#fl6['Q']=1
i=1
iq=fl6[fl6['solution'].astype(str)=='[2, 2, -3, 5, -6, 7, -8, -9, 10]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0 
iq=fl6[fl6['solution'].astype(str)=='[1, 4, 5, 5, -8, -10, -10, 13]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0 
iq=fl6[fl6['solution'].astype(str)=='[3, 3, -8, -12, -12, 13, 14, 18, -19]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0 
iq=fl6[fl6['solution'].astype(str)=='[1, -5, 8, 8, 8, -11, -12, -12, 15]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0 
iq=fl6[fl6['solution'].astype(str)=='[4, 4, -9, 11, -13, -13, 18, 18, -20]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0 
iq=fl6[fl6['solution'].astype(str)=='[2, 2, 2, -3, -3, 4, -5, -5, 6]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0 
iq=fl6[fl6['solution'].astype(str)=='[2, 2, -3, 5, -6, 7, -8, -9, 10]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0 
get_score(fl6,s='DD_massless',i=i).sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q,score
0,7,"[-12, -9]","[3, -2, 7]",108,"[4, 4, 5, -7, -8, -9, 11]",[4],0,0,0,1,10,10,10,48,-1,"[{'D→nu_R+2fi+2fj': [-7, 4, 5]}]",False,1,0.00
2,8,"[-12, -11, -9]","[-1, 0, 6]",1356,"[1, -3, -3, 5, -11, 12, 12, -13]",[12],[12],0,0,1,1,10,10,60,-1,"[{'D→nu_R+4fi': [12, -3]}, {'D→nu_R+2fi+2fj': [-11, 5, 12]}]",False,1,2.00
3,8,"[-12, -6, 6]","[8, 4, 0]",1152,"[2, -3, 7, -8, -8, 11, 14, -15]",[-8],[-8],0,0,1,1,10,10,68,-1,"[{'D→nu_R+4fi': [-8, 2]}, {'D→nu_R+2fi+2fj': [-8, -3, 7]}]",False,1,0.00
1,8,"[-12, -10, -6]","[6, 12, 6]",576,"[1, 4, 5, 5, -8, -10, -10, 13]",[-10],0,0,0,1,10,10,10,56,-1,"[{'D→nu_R+2fi+2fj': [-10, -8, 13]}, {'D→nu_R+2fi+2fj': [-10, 1, 4]}]",True,0,2.00
8,9,"[-7, -4, 6]","[3, 0, -1, -2]",6,"[3, 3, -8, -12, -12, 13, 14, 18, -19]",[-12],[-12],0,0,1,1,10,10,102,-2,"[{'D→nu_R+4fi': [-12, 3]}, {'D→nu_R+2fi+2fj': [-12, -8, 14]}]",True,0,2.00
10,9,"[-3, -2, -5]","[1, -5, -4, -2]",6,"[1, -4, -4, -5, 7, -9, 16, 23, -25]",[-4],[-4],0,0,1,1,10,10,94,-1,"[{'D→nu_R+4fi': [-4, 1]}, {'D→nu_R+2fi+2fj': [-5, -4, 7]}]",False,1,0.00
7,9,"[-5, -4, -7]","[-1, 4, 3, 1]",4,"[1, 1, 1, 4, -9, -10, -10, 11, 11]",[-10],0,"[-10, 11]",0,1,10,2,10,58,-2,"[{'D→nu_R+2fi+2fj': [-10, 1, 4]}, {'X→nu_R+2fi+2fj': [-10, 4, 11]}]",False,1,5.00
4,9,"[-6, -3, 0]","[-3, -4, -5, -4]",36,"[1, -5, 8, 8, 8, -11, -12, -12, 15]",[8],0,0,[-12],1,10,10,3,80,-1,"[{'D→nu_R+2fi+2fj': [-5, 1, 8]}, {'X→nu_R+4fi': [-12, 1]}]",True,0,2.25
5,9,"[-6, -3, -6]","[5, 0, -2, -4]",13,"[4, 4, -9, 11, -13, -13, 18, 18, -20]",[4],0,0,0,1,10,10,10,110,-3,"[{'D→nu_R+2fi+2fj': [-20, 4, 18]}, {'D→nu_R+2fi+2fj': [-13, 4, 11]}, {'D→nu_R+2fi+2fj': [-20, 11, 18]}, {'D→nu_R+2fi+2fj': [-13, 4, 18]}]",True,0,4.00
6,9,"[-6, -7, -6]","[-3, -4, -1, -5]",85,"[2, 2, 2, -3, -3, 4, -5, -5, 6]",[2],0,0,0,1,10,10,10,32,-2,"[{'D→nu_R+2fi+2fj': [-5, 2, 4]}]",True,0,4.25


In [489]:
iq=fl6[fl6['solution'].astype(str)=='[3, 3, -8, -12, -12, 13, 14, 18, -19]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
get_score(fl6,s='DM_massless',i=i).sort_values(['n','DD_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q,score
0,6,"[-20, -10]","[5, 10]",15000,"[1, 1, 1, -4, -4, 5]",0,[-4],0,0,10,1,10,10,16,-2,"[{'D→nu_R+4fi': [-4, 1]}]",False,1,3.00
1,6,"[-21, 21]","[-21, -14]",43218,"[1, -4, -4, 9, 9, -11]",0,[-4],0,0,10,1,10,10,38,-1,"[{'D→nu_R+4fi': [-4, 1]}]",False,1,2.00
2,7,"[-12, -9]","[3, -6, 3]",648,"[1, 1, -4, -4, 7, 8, -9]",0,[-4],0,0,10,1,10,10,34,-2,"[{'D→nu_R+4fi': [-4, 1]}]",False,1,2.00
4,8,"[-12, -11, -9]","[-1, 0, 6]",1356,"[1, -3, -3, 5, -11, 12, 12, -13]",[12],[12],0,0,1,1,10,10,60,-1,"[{'D→nu_R+4fi': [12, -3]}, {'D→nu_R+2fi+2fj': [-11, 5, 12]}]",False,1,2.00
5,8,"[-12, -6, 6]","[8, 4, 0]",1152,"[2, -3, 7, -8, -8, 11, 14, -15]",[-8],[-8],0,0,1,1,10,10,68,-1,"[{'D→nu_R+4fi': [-8, 2]}, {'D→nu_R+2fi+2fj': [-8, -3, 7]}]",False,1,0.00
3,8,"[-12, -9, -12]","[4, 7, -9]",2160,"[1, 1, -4, -4, -4, 12, 15, -17]",0,[-4],0,0,10,1,10,10,58,-2,"[{'D→nu_R+4fi': [-4, 1]}]",False,1,2.25
6,9,"[-7, -4, 6]","[3, 0, -1, -2]",6,"[3, 3, -8, -12, -12, 13, 14, 18, -19]",[-12],[-12],0,0,1,1,10,10,102,-2,"[{'D→nu_R+4fi': [-12, 3]}, {'D→nu_R+2fi+2fj': [-12, -8, 14]}]",True,0,2.00
8,9,"[-3, -2, -5]","[1, -5, -4, -2]",6,"[1, -4, -4, -5, 7, -9, 16, 23, -25]",[-4],[-4],0,0,1,1,10,10,94,-1,"[{'D→nu_R+4fi': [-4, 1]}, {'D→nu_R+2fi+2fj': [-5, -4, 7]}]",False,1,0.00
7,9,"[-5, -4, -2]","[2, 4, -7, -4]",50,"[1, -4, -4, -4, 12, -14, 15, 18, -20]",0,[-4],0,0,10,1,10,10,92,-1,"[{'D→nu_R+4fi': [-4, 1]}]",False,1,0.25


In [490]:
iq=fl6[fl6['solution'].astype(str)=='[2, -7, -8, -8, -8, 18, 18, 18, -25]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
get_score(fl6,s='XD_massless',i=i).sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q,score
0,8,"[-10, -5, 10]","[-10, -12, 12]",400,"[5, 5, 5, -17, -27, -27, 28, 28]",0,0,"[-27, 28]",0,10,10,1,10,142,-2,"[{'X→nu_R+2fi+2fj': [-27, -17, 28]}]",False,1,2.00
1,8,"[-9, 0, -9]","[-1, 1, -1]",324,"[3, 3, 3, -7, 17, -23, -23, 27]",0,0,[-23],0,10,10,1,10,106,-1,"[{'X→nu_R+2fi+2fj': [-23, -7, 17]}]",False,1,0.00
2,9,"[-7, 0, -2]","[-4, 4, -5, -2]",46,"[4, 7, -8, 9, -16, -16, -16, 18, 18]",[18],[-16],[18],0,3,4,1,10,112,-2,"[{'D→nu_R+2fi+2fj': [-16, 7, 18]}, {'D→nu_R+4fi': [-16, 4]}, {'X→nu_R+2fi+2fj': [-8, 7, 18]}]",False,1,0.00
4,9,"[18, 13, 6]","[-10, 6, 1, -4]",608,"[2, -7, -8, -8, -8, 18, 18, 18, -25]",0,[-8],"[-8, 18]",0,10,3,1,10,112,-1,"[{'D→nu_R+4fi': [-8, 2]}, {'X→nu_R+2fi+2fj': [-7, 2, 18]}, {'X→nu_R+2fi+2fj': [-8, -7, 2]}]",False,0,0.25
3,9,"[-6, -7, -2]","[6, -2, -1, -5]",158,"[4, 4, 4, 5, -6, -6, -6, -10, 11]",0,0,"[4, -6]",0,10,10,1,10,56,-1,"[{'X→nu_R+2fi+2fj': [-10, -6, 11]}, {'X→nu_R+2fi+2fj': [-10, 4, 11]}]",True,1,0.25


In [454]:
df=get_score(fl6,s='XM_massless',i=i).sort_values(['n','DD_massless','DM_massless','XD_massless'])
tmp6=tmp6.append(df).reset_index(drop=True)
df

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q


In [491]:
i=2
iq=fl6[fl6['solution'].astype(str)=='[1, 3, -4, 5, -6, -6, 7]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
iq=fl6[fl6['solution'].astype(str)=='[1, -3, -4, -5, 6, 6, 8, -9]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
iq=fl6[fl6['solution'].astype(str)=='[4, 4, -7, 15, 15, -16, -16, -23, 24]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
iq=fl6[fl6['solution'].astype(str)=='[1, -2, -3, -6, -7, 8, 12, 12, -15]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
iq=fl6[fl6['solution'].astype(str)=='[1, 1, 2, 2, 3, -5, -6, -6, 8]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
iq=fl6[fl6['solution'].astype(str)=='[1, -2, -2, -2, 5, -7, 8, 9, -10]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
get_score(fl6,s='DD_massless',i=i).sort_values(['n','DM_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q,score
0,6,"[-21, 14]","[-21, 3]",21168,"[1, -4, -8, 14, 14, -17]",[14],0,0,0,2,10,10,10,58,-1,"[{'D→nu_R+2fi+2fj': [-8, 1, 14]}]",False,1,0.00
3,7,"[-12, 3]","[-6, -7, 6]",135,"[7, 10, -18, 25, -28, -28, 32]",[-28],[-28],0,0,2,2,10,10,148,-1,"[{'D→nu_R+4fi': [-28, 7]}, {'D→nu_R+2fi+2fj': [-28, -18, 32]}]",False,1,0.00
1,7,"[-12, -9]","[-12, -11, -2]",351,"[2, 2, -4, 7, -8, -8, 9]",[2],[-8],0,0,2,3,10,10,40,-2,"[{'D→nu_R+2fi+2fj': [-8, 2, 7]}, {'D→nu_R+4fi': [-8, 2]}]",False,1,2.00
2,7,"[-12, -10]","[-12, -9, -5]",852,"[1, 3, -4, 5, -6, -6, 7]",[-6],0,0,0,2,10,10,10,32,-1,"[{'D→nu_R+2fi+2fj': [-6, -4, 7]}]",True,0,0.00
7,8,"[-12, 0, -12]","[-12, -3, -9]",2592,"[1, -5, -11, 15, -16, 20, 20, -24]",[20],[20],0,0,2,2,10,10,112,-1,"[{'D→nu_R+4fi': [20, -5]}, {'D→nu_R+2fi+2fj': [-11, 1, 20]}]",False,1,0.00
4,8,"[-12, -4, 12]","[-12, -6, 6]",1152,"[1, -7, -7, 17, 17, 19, -20, -20]",[-20],0,0,0,2,10,10,10,108,-3,"[{'D→nu_R+2fi+2fj': [-20, -7, 17]}]",False,1,4.00
5,8,"[-12, -9, 3]","[-3, -12, -6]",972,"[1, 2, 2, 4, -5, -5, -7, 8]",[2],0,0,0,2,10,10,10,34,-2,"[{'D→nu_R+2fi+2fj': [-5, 2, 4]}]",False,1,2.00
6,8,"[-12, -11, -7]","[6, 12, 6]",864,"[1, -3, -4, -5, 6, 6, 8, -9]",[6],0,0,0,2,10,10,10,42,-1,"[{'D→nu_R+2fi+2fj': [-4, 1, 6]}]",True,0,0.00
10,9,"[-7, -4, 6]","[-6, -5, 0, -3]",206,"[2, -3, -3, -8, -9, 12, 12, 14, -17]",[12],[12],0,0,2,2,10,10,80,-2,"[{'D→nu_R+4fi': [12, -3]}, {'D→nu_R+2fi+2fj': [-8, 2, 12]}]",False,1,2.00
12,9,"[-4, -3, -4]","[-3, -5, -3, -1]",4,"[4, 4, -7, 15, 15, -16, -16, -23, 24]",[-16],[-16],0,0,2,2,10,10,124,-2,"[{'D→nu_R+4fi': [-16, 4]}, {'D→nu_R+2fi+2fj': [-16, -7, 15]}]",True,0,4.00


In [492]:
iq=fl6[fl6['solution'].astype(str)=='[1, 1, 2, 3, -4, -4, -5, 6]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
iq=fl6[fl6['solution'].astype(str)=='[4, 4, -7, 15, 15, -16, -16, -23, 24]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
iq=fl6[fl6['solution'].astype(str)=='[1, -2, -3, -6, -7, 8, 12, 12, -15]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
get_score(fl6,s='DM_massless',i=i).sort_values(['n','DD_massless','XD_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q,score
0,7,"[-12, 3]","[-6, -7, 6]",135,"[7, 10, -18, 25, -28, -28, 32]",[-28],[-28],0,0,2,2,10,10,148,-1,"[{'D→nu_R+4fi': [-28, 7]}, {'D→nu_R+2fi+2fj': [-28, -18, 32]}]",False,1,0.00
5,8,"[-12, 0, -12]","[-12, -3, -9]",2592,"[1, -5, -11, 15, -16, 20, 20, -24]",[20],[20],0,0,2,2,10,10,112,-1,"[{'D→nu_R+4fi': [20, -5]}, {'D→nu_R+2fi+2fj': [-11, 1, 20]}]",False,1,0.00
3,8,"[-12, -9, 12]","[-2, 12, -4]",3024,"[1, 2, 2, -8, -8, 12, 15, -16]",[2],[-8],0,0,3,2,10,10,64,-1,"[{'D→nu_R+2fi+2fj': [-16, 2, 15]}, {'D→nu_R+4fi': [-8, 2]}]",True,1,2.00
2,8,"[-12, 6, -9]","[-12, -6, -12]",1944,"[4, 4, 7, 14, -16, -16, -22, 25]",[4],[-16],0,0,4,2,10,10,108,-2,"[{'D→nu_R+2fi+2fj': [-16, 4, 14]}, {'D→nu_R+4fi': [-16, 4]}]",False,1,2.00
1,8,"[-12, -8, -12]","[3, -3, -9]",5328,"[1, 1, 2, 3, -4, -4, -5, 6]",0,[-4],0,0,10,2,10,10,26,-2,"[{'D→nu_R+4fi': [-4, 1]}]",True,0,2.00
4,8,"[-12, -8, -6]","[-2, 7, 4]",696,"[1, -2, -4, -4, -4, 15, 22, -24]",0,[-4],0,0,10,2,10,10,76,-1,"[{'D→nu_R+4fi': [-4, 1]}]",False,1,0.25
6,9,"[-7, -4, 6]","[-6, -5, 0, -3]",206,"[2, -3, -3, -8, -9, 12, 12, 14, -17]",[12],[12],0,0,2,2,10,10,80,-2,"[{'D→nu_R+4fi': [12, -3]}, {'D→nu_R+2fi+2fj': [-8, 2, 12]}]",False,1,2.00
8,9,"[-4, -3, -4]","[-3, -5, -3, -1]",4,"[4, 4, -7, 15, 15, -16, -16, -23, 24]",[-16],[-16],0,0,2,2,10,10,124,-2,"[{'D→nu_R+4fi': [-16, 4]}, {'D→nu_R+2fi+2fj': [-16, -7, 15]}]",True,0,4.00
9,9,"[-7, -1, -6]","[3, 0, -2, -5]",15,"[1, -2, -3, -6, -7, 8, 12, 12, -15]",[12],[12],0,0,2,2,10,10,66,-1,"[{'D→nu_R+4fi': [12, -3]}, {'D→nu_R+2fi+2fj': [-7, 1, 12]}]",True,0,0.00
7,9,"[-6, 3, -6]","[-5, -4, 7, 2]",50,"[1, 1, 1, -4, -4, -11, 18, 26, -28]",0,[-4],0,0,10,2,10,10,94,-2,"[{'D→nu_R+4fi': [-4, 1]}]",False,1,3.00


In [493]:
iq=fl6[fl6['solution'].astype(str)=='[2, -3, -3, -3, -5, 7, 7, 8, -10]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
get_score(fl6,s='XD_massless',i=i).sort_values(['n','DD_massless','DM_massless','XM_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q,score
0,9,"[-5, -4, -7]","[-1, 4, 3, 1]",4,"[1, 1, 1, 4, -9, -10, -10, 11, 11]",[-10],0,"[-10, 11]",0,1,10,2,10,58,-2,"[{'D→nu_R+2fi+2fj': [-10, 1, 4]}, {'X→nu_R+2fi+2fj': [-10, 4, 11]}]",False,1,2
2,9,"[-5, 3, 2]","[-1, -6, 2, 4]",12,"[3, 4, -10, -10, -10, 12, 12, 13, -14]",[12],0,[12],0,2,10,2,10,88,-2,"[{'D→nu_R+2fi+2fj': [-10, 4, 12]}, {'X→nu_R+2fi+2fj': [-14, 12, 13]}]",False,1,0
1,9,"[-6, 0, 2]","[-2, -1, -3, 1]",16,"[3, 3, 3, -4, -4, 8, -11, -11, 13]",[-4],0,"[-11, -4]",0,3,10,2,10,60,-2,"[{'D→nu_R+2fi+2fj': [-11, -4, 13]}, {'X→nu_R+2fi+2fj': [-11, -4, 8]}]",False,1,2
3,9,"[16, -1, -6]","[1, 5, -5, -1]",32,"[3, -4, -4, -9, -13, 16, 16, 16, -21]",0,[16],[-4],0,10,4,2,10,102,-2,"[{'D→nu_R+4fi': [16, -4]}, {'X→nu_R+2fi+2fj': [-9, -4, 3]}]",False,1,0
4,9,"[-5, 3, -7]","[1, -3, -7, 4]",98,"[5, 7, 7, -8, -15, -15, -15, 17, 17]",0,0,"[7, 17]",0,10,10,2,10,106,-2,"[{'X→nu_R+2fi+2fj': [-8, 7, 17]}]",False,1,2
5,9,"[-7, -6, -4]","[4, -4, -1, -7]",140,"[2, -3, -3, -3, -5, 7, 7, 8, -10]",0,0,[7],0,10,10,2,10,48,-1,"[{'X→nu_R+2fi+2fj': [-10, 7, 8]}]",True,0,0


In [494]:
iq=fl6[fl6['solution'].astype(str)=='[1, 3, 3, 3, -5, -7, -7, 9]'].index[0] #unstable dark matter
fl6.loc[iq,'Q']=0
get_score(fl6,s='XM_massless',i=i).sort_values(['n','DD_massless','DM_massless','XD_massless'])

,n,l,k,gcd,solution,DD,DM,XD,XM,DD_massless,DM_massless,XD_massless,XM_massless,total,sltn_len,nu_R,unstable,Q,score
0,8,"[-12, -8, -12]","[-4, -12, 4]",4096,"[1, 3, 3, 3, -5, -7, -7, 9]",0,0,0,[-7],10,10,10,2,38,-1,"[{'X→nu_R+4fi': [-7, 1]}]",True,0,0
1,9,"[-5, -6, -7]","[-4, -3, 3, -5]",102,"[1, -4, 5, 5, -9, -9, -9, 10, 10]",0,0,0,[5],10,10,10,2,62,-1,"[{'X→nu_R+4fi': [5, 1]}]",False,1,2


In [495]:
fl6.value_counts('Q')

Q
1    34
0    18
dtype: int64

checks

In [496]:
kk=fl[fl['Q']!=0].reset_index(drop=True).copy()
kk=fl6[fl6['Q']!=0].reset_index(drop=True).copy()
for i in kk.index:
#if True:
#    i=6
    for s in ['DD','DM','XD','XM']:
        if kk.loc[i,'{}_massless'.format(s)]==0:
            kk.loc[i,'n']=add_boldsymbol(kk.loc[i,'n'])
            kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])    
#kk=fl[['n','l','k','solution','n','DD','DM','XD','XM']]
kkk=kk[['n','l','k','solution','gcd','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )